<a href="https://colab.research.google.com/github/preenet/ds_talk_nlp/blob/main/DS_talk_Intro_to_NLP_Workshop_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pree Thiengburanathum
# pree.t@cmu.ac.th
# workshop part 1
# We use wisesight sentiment corpus, can be downloaded at 
https://github.com/PyThaiNLP/wisesight-sentiment/ or 

https://www.kaggle.com/c/wisesight-sentiment/data  
Perform text classification on Thai social media posts 


train.txt - the training data. Each line is a piece of text.  
train_label.txt - the label of the training data. Each line is the label   corresponding to the same line in train.txt.  
test.txt - the test data. Each line is a piece of text.  
test_majority.csv - a sample submission file using the majority class in the correct format.  
The Id in test_majority.csv is the line number in test.txt 

no feature engineering in this notebook, teaching proposed only!  



In [78]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [79]:
import re
import pandas as pd
from matplotlib import pyplot as plt

!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
!pip install emoji
from pythainlp import word_tokenize


  Using cached https://github.com/PyThaiNLP/pythainlp/archive/dev.zip


In [80]:
import emoji
from google.colab import drive
drive.mount('/content/gdrive')
data_path = '/content/gdrive/My Drive/Colab Notebooks/WiseSight Sentiment/kaggle-competition/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [81]:
!ls '/content/gdrive/My Drive/Colab Notebooks/WiseSight Sentiment/kaggle-competition'

competition.ipynb  test_majority.csv  text_generation.ipynb  train.txt
README.md	   test_solution.csv  train_label.txt
test_label.txt	   test.txt	      train_model.py


In [82]:
# we open the given training set, namely train.txt and the target feature, namely train_label.txt
# then, we save as csv to the current directory.
features = []
targets = []

with open(data_path + "train.txt") as f:
    for line in f:
        features.append(line.strip())

with open(data_path+ "train_label.txt") as f:
    for line in f:
        targets .append(line.strip())

df = pd.DataFrame({ "targets": targets, "features": features })

df.to_csv("train.csv", index=False)
df.shape

(24063, 2)

In [83]:
# we do the sampe for testing
features = []
targets = []

with open(data_path + "test.txt") as f:
    for line in f:
        features.append(line.strip())

with open(data_path+ "test_label.txt") as f:
    for line in f:
        targets .append(line.strip())

df_test = pd.DataFrame({ "targets": targets, "features": features })

df_test.to_csv("test.csv", index=False)
df_test.shape

(2674, 2)

In [84]:
df.head(3)

,targets,features
0,neu,ประเทศเราผลิตและส่งออกยาสูบเยอะสุดในโลกจิงป่าวคับ
1,neu,คะ
2,neg,อิเหี้ยออมทำกูอยากกินเอ็มเค


In [85]:
# check for any missing vlue
df.isnull().sum()

targets     0
features    0
dtype: int64

In [86]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# try different machine-learning algorithm
#knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
logistic = LogisticRegression(C=2., penalty='l2', solver='liblinear', dual=False, multi_class='ovr')


# demonstrate w/o pipelining
coun_vect = CountVectorizer(max_features=15000)
count_matrix = coun_vect.fit_transform( df['features'] )
count_matrix_test = coun_vect.fit_transform( df_test['features'] )


# once we have the IDF values, wecan now compute the tf-idf scores for any document or set of documents.
tfidf_trans = TfidfTransformer()
vectors = tfidf_trans.fit_transform(count_matrix)
vectors_test = tfidf_trans.fit_transform(count_matrix_test)

# demonstrate using pipelining
#logis = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("tfidf", TfidfTransformer()), ("logistic",  logistic)])
count_matrix, vectors, count_matrix_test, vectors_test

(<24063x15000 sparse matrix of type '<class 'numpy.int64'>'
 	with 251593 stored elements in Compressed Sparse Row format>,
 <24063x15000 sparse matrix of type '<class 'numpy.float64'>'
 	with 251593 stored elements in Compressed Sparse Row format>,
 <2674x15000 sparse matrix of type '<class 'numpy.int64'>'
 	with 35378 stored elements in Compressed Sparse Row format>,
 <2674x15000 sparse matrix of type '<class 'numpy.float64'>'
 	with 35378 stored elements in Compressed Sparse Row format>)

In [87]:
# k-fold crossvalidtion to get generalize of model performance here.
score = cross_val_score( logistic, vectors, df['targets'].to_numpy(), cv = 5).mean()
#score = cross_val_score( logistic, df['features'].to_numpy(), df['targets'].to_numpy(), cv = 5).mean()

In [88]:
print(score)

0.6456379309362166


In [89]:
# see actual performance with test set
logistic.fit(vectors, df.targets.to_numpy())
logistic.score(vectors_test, df_test.targets.to_numpy())

0.5329094988780853

In [90]:
#lets pre-processing the features :(
# took it from https://github.com/PyThaiNLP/wisesight-sentiment/blob/master/kaggle-competition/competition.ipynb
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [91]:
# let's do it again properly with proper sampling :)
pos_df = df[df.targets == "pos"]
neu_df = df[df.targets == "neu"]
neg_df = df[df.targets== "neg"]

In [92]:
sentiment_df = pd.concat([neg_df, pos_df, neu_df])
sentiment_df = sentiment_df.reset_index(drop=True)
sentiment_df.groupby("targets").features.describe()

,count,unique,top,freq
targets,,,,
neg,6140,6129,#ERROR!,12
neu,13105,13087,#ERROR!,19
pos,4300,4300,เลิศ!ชอบมากคร่ๅๅๅ,1


In [93]:
sentiment_df['processed'] = sentiment_df.features.map(lambda x: '|'.join(process_text(x)))

In [94]:
# performing random hold-out method
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentiment_df['processed'], sentiment_df.targets, test_size=0.20, random_state=42)

In [97]:
X_train.head()

177          มัน|คือ|ฝี|หนอง|xxrep|😨|#|เจริญอาหาร|มั้ย|ล่ะ
3        ลิป|มัน|ของ|เเบร์น|mistine|ราคา|กลาง|ๆ|ไม่|แพง...
17957                                       ครับ|เสี่ย|บอส
16938    ทุกสิ่ง|อย่าง|ที่|พวก|เค้า|ต้าน|เรา|ก็|แค่|ประ...
13996    จำนวน|speed|ของ|เกียร์|ไม่|ได้|บอ|กว่า|รถ|จะ|ต...
Name: processed, dtype: object

In [103]:
tfidf = TfidfVectorizer()
tfidf_fit = tfidf.fit(sentiment_df['processed'])
text_train = tfidf_fit.transform(X_train)
text_valid = tfidf_fit.transform(X_test)
text_train.shape, text_valid.shape

((18836, 16257), (4709, 16257))

In [105]:
score = cross_val_score( logistic, text_train, y_train, cv = 5).mean()
score

0.6956892518953004